In [11]:
from statsmodels.stats.contingency_tables import mcnemar
import pandas as pd
from sklearn.metrics import confusion_matrix

In [26]:
df1 = pd.read_csv('/nfs/masi/gaor2/saved_file/CNNLung/rgnet/3split_margin/result/test_combineall_lossmin_sort.csv')
df0 = pd.read_csv('/nfs/masi/gaor2/saved_file/CNNLung/rgnet/3split_base/result/test_combineall_lossmin_sort.csv')

In [47]:
assert df0['gt_list'].tolist() == df1['gt_list'].tolist()

pred0_list = df0['prob_list'].tolist()
pred1_list = df1['prob_list'].tolist()

pred0_list = [int(i > 0.5) for i in pred0_list] #0.19678
pred1_list = [int(i > 0.5) for i in pred1_list] #0.25428


t0_0 = sum([int(pred0_list[i] == gt_list[i] and pred1_list[i] == gt_list[i]) for i in range(len(gt_list))])

t1_0 = sum([int(pred0_list[i] != gt_list[i] and pred1_list[i] == gt_list[i]) for i in range(len(gt_list))])

t0_1 = sum([int(pred0_list[i] == gt_list[i] and pred1_list[i] != gt_list[i]) for i in range(len(gt_list))])

t1_1 = sum([int(pred0_list[i] != gt_list[i] and pred1_list[i] != gt_list[i]) for i in range(len(gt_list))])

table = [[t0_0, t0_1], [t1_0, t1_1]]
print (table)

[[1529, 49], [82, 627]]


In [49]:
sum([int(pred0_list[i] == gt_list[i]) for i in range(len(gt_list))]) / len(gt_list)

0.689986882378662

In [9]:
table =  [[1529 * 2, 82 * 2], [49 * 2, 627 * 2]]
print (table)
table = [[152 *3 , 8 * 3], [4 * 3, 62 * 3]]
result = mcnemar(table, exact=True)

print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))


[[3058, 164], [98, 1254]]
statistic=12.000, p-value=0.065


In [17]:
# bootstrap
#https://github.com/mateuszbuda/ml-stat-util/blob/master/stat_util.py
import numpy as np
from scipy.stats import percentileofscore
def pvalue(
    y_true,
    y_pred1,
    y_pred2,
    score_fun,
    n_bootstraps=2000,
    two_tailed=True,
    seed=None,
    reject_one_class_samples=True,
):
    """
    Compute p-value for hypothesis that score function for model I predictions is higher than for model II predictions
    using bootstrapping.
    :param y_true: 1D list or array of labels.
    :param y_pred1: 1D list or array of predictions for model I corresponding to elements in y_true.
    :param y_pred2: 1D list or array of predictions for model II corresponding to elements in y_true.
    :param score_fun: Score function for which confidence interval is computed. (e.g. sklearn.metrics.accuracy_score)
    :param n_bootstraps: The number of bootstraps. (default: 2000)
    :param two_tailed: Whether to use two-tailed test. (default: True)
    :param seed: Random seed for reproducibility. (default: None)
    :param reject_one_class_samples: Whether to reject bootstrapped samples with only one label. For scores like AUC we
    need at least one positive and one negative sample. (default: True)
    :return: Computed p-value, array of bootstrapped differences of scores.
    """

    assert len(y_true) == len(y_pred1)
    assert len(y_true) == len(y_pred2)

    return pvalue_stat(
        y_true=y_true,
        y_preds1=y_pred1,
        y_preds2=y_pred2,
        score_fun=score_fun,
        n_bootstraps=n_bootstraps,
        two_tailed=two_tailed,
        seed=seed,
        reject_one_class_samples=reject_one_class_samples,
    )


def pvalue_stat(
    y_true,
    y_preds1,
    y_preds2,
    score_fun,
    stat_fun=np.mean,
    n_bootstraps=2000,
    two_tailed=True,
    seed=None,
    reject_one_class_samples=True,
):
    """
    Compute p-value for hypothesis that given statistic of score function for model I predictions is higher than for
    model II predictions using bootstrapping.
    :param y_true: 1D list or array of labels.
    :param y_preds1: A list of lists or 2D array of predictions for model I corresponding to elements in y_true.
    :param y_preds2: A list of lists or 2D array of predictions for model II corresponding to elements in y_true.
    :param score_fun: Score function for which confidence interval is computed. (e.g. sklearn.metrics.accuracy_score)
    :param stat_fun: Statistic for which p-value is computed. (e.g. np.mean)
    :param n_bootstraps: The number of bootstraps. (default: 2000)
    :param two_tailed: Whether to use two-tailed test. (default: True)
    :param seed: Random seed for reproducibility. (default: None)
    :param reject_one_class_samples: Whether to reject bootstrapped samples with only one label. For scores like AUC we
    need at least one positive and one negative sample. (default: True)
    :return: Computed p-value, array of bootstrapped differences of scores.
    """

    y_true = np.array(y_true)
    y_preds1 = np.atleast_2d(y_preds1)
    y_preds2 = np.atleast_2d(y_preds2)
    assert all(len(y_true) == len(y) for y in y_preds1)
    assert all(len(y_true) == len(y) for y in y_preds2)

    np.random.seed(seed)
    z = []
    for i in range(n_bootstraps):
        readers1 = np.random.randint(0, len(y_preds1), len(y_preds1))
        readers2 = np.random.randint(0, len(y_preds2), len(y_preds2))
        indices = np.random.randint(0, len(y_true), len(y_true))
        if reject_one_class_samples and len(np.unique(y_true[indices])) < 2:
            continue
        reader_scores = []
        for r in readers1:
            reader_scores.append(score_fun(y_true[indices], y_preds1[r][indices]))
        score1 = stat_fun(reader_scores)
        reader_scores = []
        for r in readers2:
            reader_scores.append(score_fun(y_true[indices], y_preds2[r][indices]))
        score2 = stat_fun(reader_scores)
        z.append(score1 - score2)

    p = percentileofscore(z, 0.0, kind="weak") / 100.0
    if two_tailed:
        p *= 2.0
    return p, z

In [13]:
import pandas as pd
df = pd.read_csv('/nfs/masi/gaor2/saved_file/Clinical_spore/0611/external/combine_ext.csv')
gt_list = df['target'].tolist() 
pred0_list = df['kaggle'].tolist() 
pred1_list = df['pred'].tolist() 

from sklearn.metrics import roc_auc_score

p, z = pvalue(gt_list, pred1_list, pred0_list, score_fun=roc_auc_score)

cnt = 0
for i in range(len(z)):
    if z[i] > 0:
        cnt += 1
print (cnt, p)

1989 0.011


In [40]:
import pandas as pd
df1 = pd.read_csv('/nfs/masi/gaor2/saved_file/Clinical_spore/multipath/0712/external/average.csv')
df0 = pd.read_csv('/nfs/masi/gaor2/saved_file/Clinical_spore/multipath/0712/onlyboth/train4fold_SGD/average.csv')

#df1 = df1.query('diag < 0 or target == 0')
#df0 = df0.query('diag < 0 or target == 0')

assert df0['BothTar'].tolist() == df1['BothTar'].tolist()
gt_list = df0['BothTar'].tolist() #* 5

pred0_list = df0['BOTHPred'].tolist()#* 5
pred1_list = df1['BOTHPred'].tolist() #* 5

from sklearn.metrics import roc_auc_score

p, z = pvalue(gt_list, pred1_list, pred0_list, score_fun=roc_auc_score)
cnt = 0
for i in range(len(z)):
    if z[i] > 0:
        cnt += 1
print (cnt, p)

1958 0.042


In [39]:
import pandas as pd
df1 = pd.read_csv('/nfs/masi/gaor2/saved_file/Clinical_spore/0622/all/mean.csv')
ID1 = df1['ID_list'].tolist()
ID1 = [i.replace('.npy', '') for i in ID1]

df = pd.read_csv('/nfs/masi/gaor2/data/Cotrain/clinical_spore/spore/norm/spore_with2yr.csv')
df = df.query('plco == plco')
df = df.loc[df['id'] != '00000981time20180709']
kaggle_pred = df['cancer'].tolist()
tar = df['gt_reg'].tolist() 
ID = df['id'].tolist()

assert ID1 == ID

pred1_list = df1['pred'].tolist() #* 5
tar1_list = df1['target'].tolist() #* 5
print (len(tar), len(tar1_list))
assert tar == tar1_list
from sklearn.metrics import roc_auc_score

p, z = pvalue(tar1_list * 5, pred1_list * 5, kaggle_pred * 5, score_fun=roc_auc_score)
cnt = 0
for i in range(len(z)):
    if z[i] > 0:
        cnt += 1
print (cnt, p)


220 220
1988 0.012


In [31]:
df1['ID_list']

0      00000009time20160420.npy
1      00000010time20140214.npy
2      00000010time20160127.npy
3      00000015time20140307.npy
4      00000015time20150713.npy
                 ...           
215    00001082time20180131.npy
216    00001104time20130130.npy
217    00001104time20140312.npy
218    00001104time20150820.npy
219    00001112time20130322.npy
Name: ID_list, Length: 220, dtype: object